In [ ]:
#%pip install qiskit --upgrade --quiet
#%pip install qiskit-nature --upgrade --quiet
#%pip install pyscf --upgrade --quiet # 1.7.6 -> 2.1.1
#%pip install qiskit_nature_pyscf --upgrade --quiet

#!pip install pennylane pennylane-qiskit --quiet # after installing pennylane-qiskit kernel need to be restarted

In [ ]:
# connect with Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


[Details of dataset](https://github.com/qulacs/Quantaggle_dataset/blob/master/datasets/Small_Molecules_1/README.md)

In [ ]:
!pip install openfermion --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 KB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 KB 11.1 MB/s eta 0:00:00


### Import molecular data from file

In [ ]:
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner

# .hdf5 file  contain information on self-consistent field (SCF) molecular orbitals and their overlaps
# interatomic distance at which the molecule is most stable is having the lowest FCI energy
distance = "1.3165" #"0.50"
molecular_data = MolecularData(filename=f"/content/gdrive/MyDrive/qhack/BeH2_sto-3g_singlet_{distance}.hdf5") # load hdf5 file
molecular_hamiltonian = get_fermion_operator(molecular_data.get_molecular_hamiltonian()) # get an instance of second quantized hamiltonian

In [ ]:
print(f"description: {molecular_data.description}", 
      "\n", f"geometry: {molecular_data.geometry}", 
      "\n", f"general_calculations: {molecular_data.general_calculations}",
      "\n", f"basis: {molecular_data.basis}",
      "\n", f"charge: {molecular_data.charge}",
      "\n", f"multiplicity: {molecular_data.multiplicity}",
      "\n", f"fci_energy: {molecular_data.fci_energy}",
      "\n", f"nuclear_repulsion: {molecular_data.nuclear_repulsion}",
      "\n", f"n_qubits: {molecular_data.n_qubits}",      
      )

description: BeH2 
 geometry: [('H', [0.0, 0.0, -1.3165]), ('Be', [0.0, 0.0, 0.0]), ('H', [0.0, 0.0, 1.3165])] 
 general_calculations: {'bond_length': 1.3165, '1st_excited_energy': -15.327119910841656} 
 basis: sto3g 
 charge: 0 
 multiplicity: 1 
 fci_energy: -15.595246750705304 
 nuclear_repulsion: 3.4166397970527917 
 n_qubits: 14


[Molecular data attributes](https://quantumai.google/reference/python/openfermion/chem/MolecularData):
- `geometry`- A list of tuples giving the coordinates of each atom. An example is [('H', (0, 0, 0)), ('H', (0, 0, 0.7414))]. Distances in angstrom. Use atomic symbols to specify atoms.
- `basis` - A string giving the basis set. An example is 'cc-pvtz'.
- `charge` - An integer giving the total molecular charge. Defaults to 0.
- `multiplicity`- An integer giving the spin multiplicity.

Definitions:
- `1st_excited_energy`: exact 1st excited state energy in the singlet (spin S^2=0) sector calculated by the full configuration interaction
- `bond_length_opt`: optimal bond length in which `fci_energy` takes minimum (4 digits accuracy after the decimal point). This value is common among data points for the same molecule.
- `fci_energy_lowest`: ground state energy at `bond_length_opt`

In [ ]:
distance = molecular_data.general_calculations["bond_length"]
distance

1.3165

In [ ]:
jw_hamiltonian = jordan_wigner(molecular_hamiltonian) # get a Pauli operator representation of the hamiltonian
#jw_hamiltonian

In [ ]:
!pip install qiskit qiskit-nature pyscf qiskit_nature_pyscf --quiet

In [ ]:
# colab requirement for qiskit draw function, kernel restart needed after install
!pip install pylatexenc --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


[Qiskit Nature 0.5: Toward quantum-centric supercomputing in the realm of natural sciences](https://medium.com/qiskit/qiskit-nature-0-5-toward-quantum-centric-supercomputing-in-the-realm-of-natural-sciences-a2fe854737cf)

In [ ]:
from pyscf import gto, scf, mcscf
import numpy as np
import qiskit
qiskit.__qiskit_version__
from qiskit.algorithms.optimizers import SLSQP, SPSA, L_BFGS_B
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.algorithms import VQEUCCFactory, GroundStateEigensolver
from qiskit_nature.second_q.mappers import QubitConverter, ParityMapper, JordanWignerMapper
from qiskit_nature_pyscf import QiskitSolver  

[Nonorthogonal configuration interaction study of the ground state of $BeH_{2}$](https://aip.scitation.org/doi/pdf/10.1063/1.1679000)

>For the largest basis, a calculated energy of −15.8478 hartree at an equilibrium Be–H bond distance of 2.52 bohr is obtained with an optimized 80‐term wavefunction

In [ ]:
#import basis_set_exchange

# Define molecule BeH2
mol = gto.M(
  #atom="Be 0 0 0; H -0.944865 0 0; H 0.944865 0 0",
  atom="H 0 0 -1.3165; Be 0 0 0; H 0 0 1.3165",
  #atom="Be 0 0 0; H 0 0 -2.52; H 0 0 2.52",
  basis="sto3g",
  verbose=4
)

System: uname_result(system='Linux', node='f75763291be3', release='5.10.147+', version='#1 SMP Sat Dec 10 16:00:40 UTC 2022', machine='x86_64', processor='x86_64')  Threads 2
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
numpy 1.22.4  scipy 1.7.3
Date: Wed Feb 22 21:42:21 2023
PySCF version 2.1.1
PySCF path  /usr/local/lib/python3.8/dist-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Be     0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000  -2.520000000000 AA    0.000000000000   0.000000000000  -4.762109833904 Bohr   0.0
[INPUT]

In [ ]:
#h_f = scf.RHF(mol).run()

h_f = scf.RHF(mol)
h_f.max_cycle = 250

#h_f.conv_tol = 1e-03

#from pyscf.scf.hf import init_guess_by_1e
#h_f.run(init_guess_by_1e(mol=mol))

h_f.run()

# https://github.com/qulacs/Quantaggle_dataset/blob/master/competitions/GS-PES/competition_GS-PES.csv
# fci_energy_lowest: -15.595246750703826



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 250
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /content/tmpv3yek7gh
max_memory 4000 MB (current use 361 MB)
Set gradient conv threshold to 3.16228e-05
init E= -14.9302120098519
  HOMO = -0.127581628631299  LUMO = -0.0348545582811847
cycle= 1 E= -15.1204480274949  delta_E= -0.19  |g|= 0.127  |ddm|= 1.46
  HOMO = -0.197623280756584  LUMO = 0.0620136701247251
cycle= 2 E= -15.1472292140917  delta_E= -0.0268  |g|= 0.0628  |ddm|= 0.431
  HOMO = -0.22472091945782  LUMO = 0.0871386409119163
cycle= 3 E= -15.1560657501581  delta_E= -0.00884  |g|= 0.013  |ddm|= 0.392
  HOMO = -0.223486204079311  LUMO = 0.0909345544297282
cycle= 4 E= -15.1562657310011  delta_E= -0.0002  |g|= 0.00119  |ddm|= 0.0564
  HOMO = -0.22514

RHF object of <class 'pyscf.scf.hf.RHF'>

In [ ]:
params_history = []
def callback(xk):
    return params_history.append(xk)

In [ ]:
#  the post-HF method to further improve our solution to the Schrödinger equation
cas = mcscf.CASSCF(h_f, 2, 2)

estimator = Estimator()

# To check
# https://github.com/Qiskit/qiskit-nature/issues/342
#quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))
#vqe = VQEUCCFactory(quantum_instance, UCCSD(), SLSQP(callback=callback)) #L_BFGS_B()) #

#vqe = VQEUCCFactory(estimator, UCCSD(), SLSQP(callback=callback)) #L_BFGS_B()) #
vqe = VQEUCCFactory(estimator, UCCSD(), L_BFGS_B(callback=callback)) #L_BFGS_B()) #

converter = QubitConverter(ParityMapper(), two_qubit_reduction=True, z2symmetry_reduction=None)
#converter = QubitConverter(JordanWignerMapper(), z2symmetry_reduction="auto")

algorithm = GroundStateEigensolver(converter, vqe)

cas.fcisolver = QiskitSolver(algorithm)

cas.run()


******** <class 'pyscf.mcscf.mc1step.CASSCF'> ********
CAS (1e+1e, 2o), ncore = 2, nvir = 3
max_cycle_macro = 50
max_cycle_micro = 4
conv_tol = 1e-07
conv_tol_grad = None
orbital rotation max_stepsize = 0.02
orbital rotation threshold for CI restart = 0.01
augmented hessian ah_max_cycle = 30
augmented hessian ah_conv_tol = 1e-12
augmented hessian ah_linear dependence = 1e-14
augmented hessian ah_level shift = 1e-08
augmented hessian ah_start_tol = 2.5
augmented hessian ah_start_cycle = 3
augmented hessian ah_grad_trust_region = 3
kf_trust_region = 3
kf_interval = 4
ci_response_space = 4
ci_grad_trust_region = 3
with_dep4 0
natorb = False
canonicalization = True
sorting_mo_energy = False
ao2mo_level = 2
chkfile = /content/tmpevrbjnxk
max_memory 4000 MB (current use 361 MB)
internal_rotation = False
CASCI E = -15.5610904914106
Set conv_tol_grad to 0.000316228
macro iter 1 (3 JK  1 micro), CASSCF E = -15.5610906187054  dE = -1.2729481e-07
               |grad[o]|=0.000541  |ddm|=9.73e-07

In [ ]:
params_history[-1]

array([-0.00737238, -0.00727464, -0.1282167 , -0.12929083])